In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor, Resize, Compose, v2
from torchvision.datasets import Cityscapes
from torch.utils.data import random_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import utils
import wandb



In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: svenbbs. Use `wandb login --relogin` to force relogin


True

In [3]:
lr_rate = 1e-3
epochs = 5
batch_size = 64
subsize = (128, 256)
momentum = 0.9
mean, std = ([0.2776, 0.3131, 0.2740]), ([0.1695, 0.1745, 0.1706])

In [4]:
augmented_transform = transforms.Compose([
    v2.ToTensor(),
    v2.Resize(subsize),
    v2.RandomHorizontalFlip(1),  # Random horizontal flip
    #v2.RandomRotation(10),  # Random rotation up to 10 degrees
    #v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color jitter
    
])

augmented_target_transform = transforms.Compose([
    v2.ToTensor(),
    v2.Resize(subsize),
    #v2.RandomHorizontalFlip(1),  # Same transformation as input images
    #v2.RandomRotation(10),  # Same transformation as input images
])

normal_transform = transforms.Compose([
    v2.ToTensor(),
    v2.Resize(subsize),
])
normal_target_transform = transforms.Compose([
    v2.ToTensor(),
    v2.Resize(subsize),
])


#Desktop
normal_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=normal_transform, target_transform=normal_target_transform)
augmented_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=augmented_transform, target_transform=augmented_target_transform)
dataset = torch.utils.data.ConcatDataset([normal_dataset, augmented_dataset])


#Laptop
#dataset = Cityscapes(root="C:/Users/20182573/Documents/CityScapes", split='train', mode='fine', target_type='semantic', transform=transform, target_transform=target_transform)

#subset_small, subset_big = random_split(dataset, [0.2,0.8])
train_dataset, val_dataset = random_split(dataset, [0.8,0.2])

<>:28: SyntaxWarning: invalid escape sequence '\C'
<>:29: SyntaxWarning: invalid escape sequence '\C'
<>:28: SyntaxWarning: invalid escape sequence '\C'
<>:29: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Sven\AppData\Local\Temp\ipykernel_18388\3415046684.py:28: SyntaxWarning: invalid escape sequence '\C'
  normal_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=normal_transform, target_transform=normal_target_transform)
C:\Users\Sven\AppData\Local\Temp\ipykernel_18388\3415046684.py:29: SyntaxWarning: invalid escape sequence '\C'
  augmented_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=augmented_transform, target_transform=augmented_target_transform)
c:\Users\Sven\anaconda3\envs\torch\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [6]:
def mean_and_std(loader):
    mean = 0.
    std = 0.
    for images, _ in loader:
        batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)

    mean /= len(loader.dataset)
    std /= len(loader.dataset)
    return mean, std

#mean, std = mean_and_std(train_dataloader)

In [7]:

from model import Model
model = Model().cuda()


loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)


In [8]:
def train(dataloader, model, loss_fn, optimizer, run):
    """
    Train a model for 1 epoch.

    Params:
    - dataloader:   dataset to train on.
    - model:        the model object to be trained.
    - loss_fn:      the loss function.
    - optimizer:    the desired optimization.
    """
    size = len(dataloader.dataset)
    model.train() #Set the model to train mode
    for batch, (IMG,SEGM) in enumerate(dataloader):
        IMG = IMG.to('cuda')
        SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
        SEGM = utils.map_id_to_train_id(SEGM).to('cuda')
        
        #predict
        pred = model(IMG)
        #Loss
        loss = loss_fn(pred, SEGM)
        

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #print loss during training
        loss, current = loss.item(), (batch + 1) * len(IMG)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        run.log({"train_loss": loss})
    
    return run
            


In [9]:
def test(dataloader, model, loss_fn):
    """
    Test a model.

    Params:
    - dataloader:   dataset to test on.
    - model:        the model object to be tested.
    - loss_fn:      the loss function.
    """
    num_batches = len(dataloader)
    model.eval() #model in eval mode
    test_loss = 0
    with torch.no_grad():
        for _, (IMG,SEGM) in enumerate(dataloader):
            IMG = IMG.to('cuda')
            SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
            SEGM = utils.map_id_to_train_id(SEGM).to('cuda')

            pred = model(IMG)
            test_loss += loss_fn(pred, SEGM).item()
            
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    return test_loss
    


In [10]:



run = wandb.init(
    # Set the project where this run will be logged
    project="CS_challenge", name="1.7",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": {lr_rate},
        "epochs": {epochs},
        "Momentum": {momentum},
        "Batch_size": {batch_size},
        "model version": 0.0,
        "subset size [%]": 100,
        "resize": {subsize},
        "Normalized": False,
        "Optimizer": 'Adam',
    },
)




for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    run = train(train_dataloader, model, loss_fn, optimizer, run)
    test_loss = test(val_dataloader, model, loss_fn)
    run.log({"Test_loss": test_loss, "Epoch": t})
print("Done!")


Epoch 1
-------------------------------
loss: 3.534487  [   64/ 4760]
loss: 3.180811  [  128/ 4760]
loss: 2.953079  [  192/ 4760]
loss: 2.812852  [  256/ 4760]
loss: 2.636143  [  320/ 4760]
loss: 2.513393  [  384/ 4760]
loss: 2.441398  [  448/ 4760]
loss: 2.337048  [  512/ 4760]
loss: 2.282129  [  576/ 4760]
loss: 2.223753  [  640/ 4760]
loss: 2.131212  [  704/ 4760]
loss: 2.099603  [  768/ 4760]
loss: 2.055564  [  832/ 4760]
loss: 1.993006  [  896/ 4760]
loss: 1.916375  [  960/ 4760]
loss: 1.874338  [ 1024/ 4760]
loss: 1.885856  [ 1088/ 4760]
loss: 1.806984  [ 1152/ 4760]
loss: 1.783181  [ 1216/ 4760]
loss: 1.744170  [ 1280/ 4760]
loss: 1.643790  [ 1344/ 4760]
loss: 1.601516  [ 1408/ 4760]
loss: 1.682422  [ 1472/ 4760]
loss: 1.635722  [ 1536/ 4760]
loss: 1.640763  [ 1600/ 4760]
loss: 1.578774  [ 1664/ 4760]
loss: 1.576934  [ 1728/ 4760]
loss: 1.538259  [ 1792/ 4760]
loss: 1.568531  [ 1856/ 4760]
loss: 1.516172  [ 1920/ 4760]
loss: 1.458884  [ 1984/ 4760]
loss: 1.430593  [ 2048/ 4760]


KeyboardInterrupt: 

In [ ]:
run.finish()

In [ ]:
torch.save(model.state_dict(), "model.pth")